In [ ]:
# ─────────────────────────────────────────────────────────────
# Block 0 · Imports & global configuration
# ─────────────────────────────────────────────────────────────
import math
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility ─────────────────────────────────────────────
SEED = 42
np.random.seed(SEED)

# Dataset location ────────────────────────────────────────────
FILE_PATH = r'D:\IITR DS Final Year Thesis\Dataset Superstore\archive\generated_single_product_dataset_with_seasonal_variation.csv'

# Economic constants (update later if you have product‑specific info)
COST_PER_UNIT_DEFAULT = 0.0        # will be overwritten by df['cost'].mean()
PRICE_GRID_SIZE       = 200        # resolution when searching for optimal price
TANH_SCALE            = 150        #   p / TANH_SCALE inside tanh(·)

print("✔️  Block 0 ready – libraries imported, seed fixed.")


# ─────────────────────────────────────────────────────────────
# Block 1 · Data loading & feature engineering
# ─────────────────────────────────────────────────────────────
# • Reads the CSV at FILE_PATH
# • Parses & sorts dates
# • Adds weekly dummies, annual sin/cos
# • Builds non‑linear price bases: price_sq, log_price, tanh_price
# • Prepares feature‑name lists for later blocks
# ─────────────────────────────────────────────────────────────

# 1) Load -----------------------------------------------------------------
df = pd.read_csv(FILE_PATH)
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.sort_values('date').reset_index(drop=True)

# 2) Basic sanity checks --------------------------------------------------
assert df['price_paid'].notna().all(), "Missing price data!"
df.rename(columns={'price_paid': 'price'}, inplace=True)   # convenience alias

# 3) Seasonality indices --------------------------------------------------
df['day_of_year'] = df['date'].dt.dayofyear
df['day_of_week'] = df['date'].dt.weekday         # Monday=0 … Sunday=6

# 3a) Weekly dummies (baseline = Sunday, i.e. day 6)
weekly_dummies = pd.get_dummies(df['day_of_week'], prefix='dow', drop_first=True)
df = pd.concat([df, weekly_dummies], axis=1)
weekly_cols = weekly_dummies.columns.tolist()

# 3b) Annual Fourier terms
df['sin_annual'] = np.sin(2 * np.pi * df['day_of_year'] / 365.0)
df['cos_annual'] = np.cos(2 * np.pi * df['day_of_year'] / 365.0)

# 4) Price‑basis functions -----------------------------------------------
df['price_sq']   = df['price'] ** 2
df['log_price']  = np.log(df['price'])
df['tanh_price'] = np.tanh(df['price'] / TANH_SCALE)

price_basis_cols = ['price', 'price_sq', 'log_price', 'tanh_price']

# 5) Feature‑name lists ---------------------------------------------------
static_feature_cols = weekly_cols + ['sin_annual', 'cos_annual']
feature_cols        = ['intercept'] + static_feature_cols + price_basis_cols

# 6) Derived globals ------------------------------------------------------
MIN_PRICE, MAX_PRICE = df['price'].min(), df['price'].max()
COST_PER_UNIT        = df['cost'].mean() if 'cost' in df.columns else COST_PER_UNIT_DEFAULT

# 7) Quick printout -------------------------------------------------------
print("DataFrame shape :", df.shape)
print("Feature columns  :", feature_cols)
print(f"Price range     : {MIN_PRICE:.2f} – {MAX_PRICE:.2f}")
print(f"Avg. unit cost  : {COST_PER_UNIT:.2f}")
print(df.head(3))

print("✔️  Block 1 complete – dataset ready & features engineered.")


# ─────────────────────────────────────────────────────────────
# Block 2 · Prior specification & initial posterior
# ─────────────────────────────────────────────────────────────
# Assumes Block 0 and Block 1 have run and defined:
#   • df, feature_cols, static_feature_cols, price_basis_cols
#   • WEEKLY_COLS etc.
# Produces:
#   • mu0, Sigma0           – prior mean / covariance
#   • sigma2                – obs‑noise variance
#   • post_mean, post_cov   – initial posterior (equal to prior)
#   • feat_idx              – dict: feature name ➜ index (handy later)
#   • price_term_indices    – list of indices for price‑basis coeffs
# ─────────────────────────────────────────────────────────────

# 1) Feature index helper ------------------------------------
feat_idx = {name: i for i, name in enumerate(feature_cols)}

# For convenience: indices of price‑related terms
price_term_indices = [feat_idx[col] for col in price_basis_cols]

# 2) Prior mean vector (μ₀) -----------------------------------
mu0 = np.zeros(len(feature_cols))

# Intercept prior: start near median quantity
mu0[feat_idx['intercept']] = df['quantity'].median()

# Monotonic priors on price terms (negative to enforce ↓ demand)
mu0[feat_idx['price']]       = -1.0      # linear price slope
mu0[feat_idx['price_sq']]    = -0.05     # gentle concavity
mu0[feat_idx['log_price']]   = -1.0
mu0[feat_idx['tanh_price']]  = -1.0

# Seasonal terms → mean 0 (no initial bias)

# 3) Prior standard deviations (σ₀) ---------------------------
prior_sd = np.ones(len(feature_cols)) * 10.0          # default weak
prior_sd[feat_idx['intercept']] = 20.0                # looser on level
# Weekly dummies & annual sin/cos: moderate prior
for col in static_feature_cols:
    prior_sd[feat_idx[col]] = 5.0
# Price terms: a bit tighter to keep monotonicity strong
for idx in price_term_indices:
    prior_sd[idx] = 2.0

# 4) Prior covariance (Σ₀) ------------------------------------
Sigma0 = np.diag(prior_sd ** 2)        # diagonal (independent priors)

# 5) Observation noise variance (σ²) --------------------------
sigma_hat = df['quantity'].std()
sigma2    = sigma_hat ** 2

# 6) Initialise posterior = prior -----------------------------
post_mean = mu0.copy()
post_cov  = Sigma0.copy()

# 7) Sanity summary -------------------------------------------
import pandas as pd
print("\n──────── Prior mean (μ₀) ────────")
print(pd.Series(mu0, index=feature_cols).round(3))
print("\n──────── Prior SD (σ₀) ──────────")
print(pd.Series(prior_sd, index=feature_cols).round(3))
print(f"\nObservation noise σ²  ≈ {sigma2:.2f}")
print(f"Feature count         = {len(feature_cols)}")
print("✔️  Block 2 ready – posterior initialised.")

# 8) Export key objects for later blocks ----------------------
# (They remain in memory; nothing to return explicitly.)


# ─────────────────────────────────────────────────────────────
# Block 3 · Utility helpers
# ─────────────────────────────────────────────────────────────
# These pure‑Python helpers keep the main loop (Block 4) tidy.
#
# Assumes Blocks 0–2 defined globals:
#   • feature_cols, static_feature_cols, price_basis_cols
#   • weekly_cols, MIN_PRICE, MAX_PRICE, PRICE_GRID_SIZE
#   • TANH_SCALE, COST_PER_UNIT, sigma2
#
# Exposes:
#   build_static_vector(row)        – fixed context part (no price)
#   price_basis(price)              – non‑linear price features
#   make_feature_vector(row, p)     – full φ vector for any price
#   blr_update(mu, Sigma, x, y)     – Kalman update (BLR)
#   sample_theta(mu, Sigma)         – θ ~ N(μ, Σ)
#   demand_pred(theta, static, p)   – deterministic demand (no noise)
#   find_optimal_price(theta, static) – argmax_p (p−c)·vθ(p)
#   clamp(val, lo, hi)              – scalar clamp
# ─────────────────────────────────────────────────────────────

import numpy as np
import math

# 1)  Context (static) vector builder ------------------------
def build_static_vector(row):
    """
    Return the intercept + weekly dummies + annual sin/cos
    for *this transaction’s date* (price terms excluded).
    Result length = 1 + len(static_feature_cols) − (dropping price bases).
    """
    vec = [1.0]                                       # intercept
    # weekly dummies in the same order as weekly_cols
    vec.extend([row[col] for col in weekly_cols])
    # annual fourier
    vec.append(row['sin_annual'])
    vec.append(row['cos_annual'])
    return np.asarray(vec, dtype=float)               # shape (len(static)+1,)

# 2)  Price‑basis builder ------------------------------------
def price_basis(price):
    """
    Compute non‑linear transformations of price.
    Order must match price_basis_cols.
    """
    return np.asarray([
        price,
        price**2,
        math.log(price),
        math.tanh(price / TANH_SCALE)
    ], dtype=float)

# 3)  Full feature vector ------------------------------------
def make_feature_vector(row, price):
    """
    Concatenate static context vector and price bases.
    Length = len(feature_cols)
    """
    static_vec = build_static_vector(row)
    return np.concatenate([static_vec, price_basis(price)])

# 4)  Kalman / BLR one‑step update ---------------------------
def blr_update(mu, Sigma, x_vec, y_obs, sigma2=sigma2):
    """
    Posterior update for Bayesian linear regression with
    known Gaussian noise variance.
    Args
      mu     – prior mean        (d,)
      Sigma  – prior covariance  (d,d)
      x_vec  – feature vector    (d,)
      y_obs  – observed target   (scalar)
    Returns
      new_mu, new_Sigma
    """
    x = x_vec.reshape(-1, 1)                        # (d,1)
    s = float(x.T @ Sigma @ x + sigma2)             # predictive variance
    K = (Sigma @ x) / s                             # Kalman gain (d,1)
    residual = y_obs - float(mu @ x_vec)            # innovation
    new_mu = mu + K.flatten() * residual
    new_Sigma = Sigma - K @ (x.T @ Sigma)
    # Symmetrise to avoid numeric drift
    new_Sigma = 0.5 * (new_Sigma + new_Sigma.T)
    return new_mu, new_Sigma

# 5)  Posterior sample ---------------------------------------
def sample_theta(mu, Sigma):
    """Draw θ ~ N(μ, Σ)."""
    return np.random.multivariate_normal(mu, Sigma)

# 6)  Deterministic demand prediction ------------------------
def demand_pred(theta, static_vec, price):
    """vθ(p) without observation noise (can be negative early)."""
    return float(theta[:-len(price_basis_cols)] @ static_vec + theta[-len(price_basis_cols):] @ price_basis(price))

# 7)  Optimal price for a sampled θ --------------------------
def find_optimal_price(theta, static_vec,
                       price_bounds=(MIN_PRICE, MAX_PRICE),
                       grid_size=PRICE_GRID_SIZE,
                       cost=COST_PER_UNIT):
    """
    Grid‑search argmax_p (p−c)·vθ(p) within [min,max].
    Returns the best price p* for this θ and context.
    """
    prices = np.linspace(price_bounds[0], price_bounds[1], grid_size)
    best_p, best_val = prices[0], -np.inf
    for p in prices:
        q = demand_pred(theta, static_vec, p)
        if q < 0:           # enforce non‑negative demand estimate
            q = 0
        profit = (p - cost) * q
        if profit > best_val:
            best_val, best_p = profit, p
    return best_p

# 8)  Scalar clamp convenience -------------------------------
def clamp(val, lo, hi):
    return max(lo, min(hi, val))

print("✔️  Block 3 loaded – helper functions ready.")



# ─────────────────────────────────────────────────────────────
# Block 4 · Online Thompson‑Sampling loop
# ─────────────────────────────────────────────────────────────
# Sequentially processes every transaction in df (chronological),
# updating the BLR posterior after *each* observation.
#
# For each row i:
#   1. Sample θᵢ ~ N(μᵢ, Σᵢ)      ← Thompson step
#   2. Optimise (p−c)·vθᵢ(p) over a grid → p*
#   3. Log p*  (recommendation)            (simulation only)
#   4. Use the row's historical (price, qty) as the environment
#      outcome (offline replay).
#   5. Kalman‑update posterior with that (x, y).
#   6. Record all diagnostics for later plots.
#
# Output (in‑memory):
#   logs  – dict of numpy arrays  (len = N transactions)
#   post_mean_final, post_cov_final – final posterior after last obs
# ─────────────────────────────────────────────────────────────

from tqdm import tqdm
import numpy as np

# 0) Prepare numpy arrays for speed ---------------------------------------
N = len(df)

rec_price         = np.empty(N)                    # Thompson‑recommended
act_price         = df['price'].values             # historical price
act_qty           = df['quantity'].values          # observed demand
pred_qty          = np.empty(N)                    # μᵢ prediction before update
profit            = np.empty(N)
cum_profit        = np.empty(N)
beta_price_mean   = np.empty(N)                    # posterior mean of *linear* price term
theta_mean_trace  = np.empty((N, len(feature_cols)))  # optional full trace

# 1) Initial posterior (from Block 2) -------------------------------------
mu  = post_mean.copy()
Sig = post_cov.copy()

# 2) Online loop ----------------------------------------------------------
for i, row in tqdm(df.iterrows(), total=N, ncols=80,
                   desc="Online BLR update"):

    # Static context vector (no price terms) for this transaction
    static_vec = build_static_vector(row)

    # 2.a Thompson‑sampling draw
    theta_sample = sample_theta(mu, Sig)

    # 2.b Recommend price p* (grid search)
    p_star = find_optimal_price(theta_sample, static_vec)
    rec_price[i] = p_star

    # 2.c One‑step‑ahead demand prediction at *actual* historical price
    x_pred = make_feature_vector(row, row['price'])
    pred_qty[i] = max(mu @ x_pred, 0.0)

    # 2.d Profit using historical price (offline replay)
    #     If 'cost' column exists use per‑row cost, else global constant
    cost_i  = row['cost'] if 'cost' in df.columns else COST_PER_UNIT
    profit_i = (row['price'] - cost_i) * row['quantity']
    profit[i] = profit_i
    cum_profit[i] = profit_i if i == 0 else cum_profit[i-1] + profit_i

    # 2.e Posterior update with (x_obs, y_obs)
    x_obs = x_pred                               # same vector
    y_obs = row['quantity']
    mu, Sig = blr_update(mu, Sig, x_obs, y_obs)

    # 2.f Logs for diagnostics
    beta_price_mean[i]  = mu[feat_idx['price']]
    theta_mean_trace[i] = mu                     # optional full trace

# 3) Final posterior snapshot ---------------------------------------------
post_mean_final = mu.copy()
post_cov_final  = Sig.copy()

print(f"\nFinal β_price mean   : {post_mean_final[feat_idx['price']]:.4f}")
print(f"Total cumulative profit: {cum_profit[-1]:.2f}")

# 4) Pack logs dict --------------------------------------------------------
logs = {
    "time"                : df['date'].values,          # numpy datetime64
    "actual_price"        : act_price,
    "recommended_price"   : rec_price,
    "actual_quantity"     : act_qty,
    "predicted_quantity"  : pred_qty,
    "profit"              : profit,
    "cum_profit"          : cum_profit,
    "beta_price_mean"     : beta_price_mean,
    # heavy but nice to have:
    "theta_mean_trace"    : theta_mean_trace,
    # final posterior:
    "post_mean_final"     : post_mean_final,
    "post_cov_final"      : post_cov_final,
    "feature_cols"        : feature_cols,
    "price_bounds"        : (MIN_PRICE, MAX_PRICE),
    "sigma2"              : sigma2,
    "cost_per_unit"       : COST_PER_UNIT,
}

print("✔️  Block 4 complete – online learning finished and logs ready.")



# ─────────────────────────────────────────────────────────────
# Block 5 · Persist results to disk
# ─────────────────────────────────────────────────────────────
# Saves everything created in Block 4 so you can reload later
# without rerunning the online loop.
#
# Creates a folder:
#       results/phase1_<YYYYMMDD_HHMMSS>/
# Inside you’ll find:
#   • transaction_logs.csv        – per‑tx price/qty/profit timeline
#   • theta_mean_trace.npz        – (N, d) posterior mean at each step
#   • beta_mean_final.npy         – final μ_T
#   • beta_cov_final.npy          – final Σ_T
#   • meta.json                   – small metadata file
# ─────────────────────────────────────────────────────────────

import os, json, pickle, numpy as np, pandas as pd
from datetime import datetime

# 1) Confirm logs exists -------------------------------------------------
try:
    logs
except NameError:
    raise RuntimeError("logs dict not found – run Block 4 first.")

# 2) Make results directory ------------------------------------------------
timestamp   = datetime.now().strftime("%Y%m%d_%H%M%S")
base_dir    = os.path.join("results", f"phase1_{timestamp}")
os.makedirs(base_dir, exist_ok=True)

# 3) Transaction‑level CSV -------------------------------------------------
df_logs = pd.DataFrame({
    "time"               : logs["time"],
    "actual_price"       : logs["actual_price"],
    "recommended_price"  : logs["recommended_price"],
    "actual_quantity"    : logs["actual_quantity"],
    "predicted_quantity" : logs["predicted_quantity"],
    "profit"             : logs["profit"],
    "cum_profit"         : logs["cum_profit"],
    "beta_price_mean"    : logs["beta_price_mean"],
})
csv_path = os.path.join(base_dir, "transaction_logs.csv")
df_logs.to_csv(csv_path, index_label="transaction_index")
print(f"✓ Saved per‑transaction log → {csv_path}")

# 4) θ‑trace (posterior mean at each step) ---------------------------------
theta_path = os.path.join(base_dir, "theta_mean_trace.npz")
np.savez_compressed(theta_path, theta=logs["theta_mean_trace"])
print(f"✓ Saved theta_mean_trace → {theta_path}")

# 5) Final posterior arrays -----------------------------------------------
np.save(os.path.join(base_dir, "beta_mean_final.npy"), logs["post_mean_final"])
np.save(os.path.join(base_dir, "beta_cov_final.npy"),  logs["post_cov_final"])
print("✓ Saved final posterior μ & Σ (.npy files)")

# 6) Metadata --------------------------------------------------------------
meta = {
    "timestamp"      : timestamp,
    "feature_cols"   : logs["feature_cols"],
    "sigma2"         : float(logs["sigma2"]),
    "price_bounds"   : [float(x) for x in logs["price_bounds"]],
    "cost_per_unit"  : float(logs["cost_per_unit"]),
    "n_transactions" : int(len(logs["actual_price"])),
}
with open(os.path.join(base_dir, "meta.json"), "w") as f:
    json.dump(meta, f, indent=2)
print("✓ Saved meta.json")

# 7) Helper loader for future notebooks -----------------------------------
def load_phase1_results(folder_path):
    """
    Quick re‑loader for everything saved by Block 5.
    Returns (df_logs, theta_trace, mu_final, Sigma_final, meta)
    """
    df_logs = pd.read_csv(os.path.join(folder_path, "transaction_logs.csv"),
                          parse_dates=["time"],
                          index_col="transaction_index")
    theta   = np.load(os.path.join(folder_path, "theta_mean_trace.npz"))["theta"]
    mu_fin  = np.load(os.path.join(folder_path, "beta_mean_final.npy"))
    Sig_fin = np.load(os.path.join(folder_path, "beta_cov_final.npy"))
    with open(os.path.join(folder_path, "meta.json"), "r") as f:
        meta = json.load(f)
    return df_logs, theta, mu_fin, Sig_fin, meta

print("\n✔️  Block 5 complete – results persisted & loader defined.")
print(f"   Folder: {base_dir}")



# ─────────────────────────────────────────────────────────────
# Block 6 · Visualisation suite
# ─────────────────────────────────────────────────────────────
# Creates publication‑quality figures:
#   6‑a Demand curve ±95 % CI vs. observed data
#   6‑b Price‑quantity scatter + fitted curve
#   6‑c Cumulative profit trajectory
#   6‑d Posterior mean of β_price over time
#   6‑e Histogram of one‑step residuals
#
# The block works in TWO modes:
#   • In‑memory  – if logs dict (from Block 4) is present
#   • Off‑disk   – provide results_folder pointing to a
#                  results/phase1_YYYYMMDD_HHMMSS directory
#
# Figures are saved into <plots_dir> and also displayed.
# ─────────────────────────────────────────────────────────────

import os, json, numpy as np, pandas as pd
import matplotlib.pyplot as plt

# 0) Choose data source ----------------------------------------------------
results_folder = None   # ← set to a saved folder OR leave None to use in‑RAM logs

if results_folder is None:
    try:
        logs
    except NameError:
        raise RuntimeError("No in‑memory logs found. "
                           "Either run Blocks 4‑5 or set results_folder.")
    df_logs       = pd.DataFrame({
        "time"              : logs["time"],
        "actual_price"      : logs["actual_price"],
        "recommended_price" : logs["recommended_price"],
        "actual_quantity"   : logs["actual_quantity"],
        "predicted_quantity": logs["predicted_quantity"],
        "profit"            : logs["profit"],
        "cum_profit"        : logs["cum_profit"],
        "beta_price_mean"   : logs["beta_price_mean"],
    })
    theta_mean_trace = logs["theta_mean_trace"]
    mu_final         = logs["post_mean_final"]
    Sigma_final      = logs["post_cov_final"]
    price_bounds     = logs["price_bounds"]
    plots_dir        = os.path.join("results", "plots_inline")
else:
    # Loader from Block 5
    df_logs, theta_mean_trace, mu_final, Sigma_final, meta = load_phase1_results(results_folder)
    price_bounds = meta["price_bounds"]
    plots_dir    = os.path.join(results_folder, "plots")

os.makedirs(plots_dir, exist_ok=True)

min_p, max_p = price_bounds
N            = len(df_logs)

# Helper: feature vector for baseline context (Sunday, avg annual)
baseline_static = np.zeros(len(static_feature_cols) + 1)  # +1 for intercept
baseline_static[0] = 1.0
def demand_curve(mu, p_grid):
    """
    Compute posterior mean demand for baseline context.
    """
    # theta split: [static coeffs | price coeffs]
    static_coeffs  = mu[:-len(price_basis_cols)]
    price_coeffs   = mu[-len(price_basis_cols):]
    demand = []
    for p in p_grid:
        basis = price_basis(p)
        demand.append(static_coeffs @ baseline_static + price_coeffs @ basis)
    return np.array(demand)

# ------------------------------------------------------------
# 6‑a Demand curve with 95 % credible interval
# ------------------------------------------------------------
p_grid = np.linspace(min_p, max_p, 300)
d_mean = demand_curve(mu_final, p_grid)
# Credible interval (variance per point)
var_vec = []
for p in p_grid:
    basis = price_basis(p)
    phi   = np.concatenate([baseline_static, basis])
    var_vec.append(phi @ Sigma_final @ phi)
var_vec = np.array(var_vec)
d_std   = np.sqrt(var_vec)
plt.figure(figsize=(7,4))
plt.plot(p_grid, d_mean, label="Posterior mean demand")
plt.fill_between(p_grid, d_mean - 2*d_std, d_mean + 2*d_std,
                 alpha=0.3, label="95% credible interval")
plt.xlabel("Price")
plt.ylabel("Quantity")
plt.title("Estimated demand curve (baseline context)")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "demand_curve_ci.png"), dpi=150)
plt.show()

# ------------------------------------------------------------
# 6‑b Scatter of data + fitted curve
# ------------------------------------------------------------
plt.figure(figsize=(7,4))
plt.scatter(df_logs["actual_price"], df_logs["actual_quantity"],
            s=10, alpha=0.25, label="Observed transactions")
plt.plot(p_grid, d_mean, linewidth=2, label="Posterior mean")
plt.xlabel("Price")
plt.ylabel("Quantity")
plt.title("Observed data vs. fitted demand curve")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "scatter_fit.png"), dpi=150)
plt.show()

# ------------------------------------------------------------
# 6‑c Cumulative profit
# ------------------------------------------------------------
plt.figure(figsize=(7,4))
plt.plot(df_logs["cum_profit"].values)
plt.xlabel("Transaction index")
plt.ylabel("Cumulative profit")
plt.title("Cumulative profit over time (historical prices)")
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "cum_profit.png"), dpi=150)
plt.show()

# ------------------------------------------------------------
# 6‑d Posterior mean β_price evolution
# ------------------------------------------------------------
plt.figure(figsize=(7,4))
plt.plot(df_logs["beta_price_mean"].values)
plt.axhline(0, linewidth=0.8)
plt.xlabel("Transaction index")
plt.ylabel("Posterior mean β_price")
plt.title("Convergence of linear price coefficient")
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "beta_price_trace.png"), dpi=150)
plt.show()

# ------------------------------------------------------------
# 6‑e Residual histogram
# ------------------------------------------------------------
residuals = df_logs["actual_quantity"].values - df_logs["predicted_quantity"].values
plt.figure(figsize=(6,4))
plt.hist(residuals, bins=50, edgecolor="k", alpha=0.8)
plt.xlabel("Residual (actual − predicted)")
plt.ylabel("Frequency")
plt.title("Distribution of 1‑step residuals")
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "residual_hist.png"), dpi=150)
plt.show()

print(f"✔️  Block 6 finished – plots saved to {plots_dir}")
print("   Files:")
for f in os.listdir(plots_dir):
    print("   •", f)


# ─────────────────────────────────────────────────────────────
# Compute final‑posterior optimal price (baseline context)
# ─────────────────────────────────────────────────────────────
import numpy as np

# 1)  Baseline context: intercept = 1, Sunday (all dummies 0), average season
baseline_static = np.zeros(len(static_feature_cols) + 1)   # +1 for intercept
baseline_static[0] = 1.0

# Split posterior mean into static / price blocks
n_static = len(baseline_static)
static_coef = mu_final[:n_static]                 # β for intercept + seasonality
price_coef  = mu_final[n_static:]                 # β for price bases

# 2)  Vectorised demand‑mean function ------------------------
def demand_mean_vec(p_array):
    """
    Posterior‑mean demand for a NumPy array of prices, baseline context.
    p_array: shape (m,)
    Returns   shape (m,)
    """
    p = np.asarray(p_array)
    # Build price‑basis matrix   shape (m, 4)
    eps = 1e-12                 # avoid log(0)
    PB = np.stack([
        p,
        p ** 2,
        np.log(p + eps),
        np.tanh(p / TANH_SCALE)
    ], axis=1)
    const_part = baseline_static @ static_coef    # scalar
    return const_part + PB @ price_coef           # vector

# 3)  Profit grid search ------------------------------------
grid = np.linspace(MIN_PRICE, MAX_PRICE, 2000)    # dense grid
demand_hat = np.maximum(demand_mean_vec(grid), 0) # clip negative preds
exp_profit = (grid - COST_PER_UNIT) * demand_hat

idx_best   = int(np.argmax(exp_profit))
opt_price  = grid[idx_best]
opt_demand = demand_hat[idx_best]
opt_profit = exp_profit[idx_best]

# 4)  Display results ---------------------------------------
print(f"──────── Optimal price from final posterior (baseline context) ────────")
print(f"Optimal price  : {opt_price:,.2f}")
print(f"Exp. demand    : {opt_demand:,.2f} units/tx")
print(f"Exp. profit/tx : {opt_profit:,.2f}")



# ─────────────────────────────────────────────────────────────
# Block 7 · Figure‑4‑style Thompson‑Sampling illustration
# ─────────────────────────────────────────────────────────────
# Recreates the three‑panel conceptual figure from the PVD‑B
# paper for *any* transaction index τ:
#   1. Posterior mean demand ±95 % CI (black)
#   2. One Thompson‑sample realisation v_TS(p)   (green)
#   3. Profit curve (p−c)·v_TS(p)                (red)
#   4. Vertical line at p*τ  (argmax of profit curve)
#
# Usage:
#     plot_ts_realisation(idx=5000, use_final_posterior=False)
#
# If use_final_posterior=True, the demand envelope uses the
# final posterior; otherwise it uses the posterior *after* the
# specified tx (mu = theta_mean_trace[idx]).
#
# NOTE:  We did not store Σ_t at every step (memory).  We use
#        the *final* covariance as a proxy for the envelope.
#        If you saved Σ_t per iteration, replace the proxy.
# ─────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt

def plot_ts_realisation(idx,
                        price_bounds=(MIN_PRICE, MAX_PRICE),
                        grid_size=400,
                        use_final_posterior=False,
                        context="row"):        # "row" | "baseline"
    """
    idx : 0‑based transaction index τ
    context:
        • "row"      → use the actual context of transaction τ
        • "baseline" → Sunday + average season (same as Block 6 curve)
    """
    # 1) Choose posterior parameters --------------------------
    if use_final_posterior:
        mu = mu_final
        Sig= Sigma_final
    else:
        mu = theta_mean_trace[idx]
        Sig= Sigma_final          # proxy for uncertainty envelope

    # 2) Context static vector -------------------------------
    if context == "baseline":
        static_vec = baseline_static
    else:
        row = df.iloc[idx]
        static_vec = build_static_vector(row)

    # 3) Build price grid ------------------------------------
    p_grid = np.linspace(price_bounds[0], price_bounds[1], grid_size)

    # 3a Posterior mean demand
    static_coef = mu[:len(static_vec)]
    price_coef  = mu[len(static_vec):]
    PB          = np.stack([
        p_grid,
        p_grid**2,
        np.log(p_grid),
        np.tanh(p_grid / TANH_SCALE)
    ], axis=1)
    d_mean = static_vec @ static_coef + PB @ price_coef

    # 3b 95 % CI using proxy Σ
    var_vec = []
    for p in p_grid:
        phi = np.concatenate([static_vec,
                              [p, p**2, np.log(p), np.tanh(p / TANH_SCALE)]])
        var_vec.append(phi @ Sig @ phi)
    d_std = np.sqrt(np.maximum(var_vec, 0))
    upper, lower = d_mean + 2*d_std, d_mean - 2*d_std

    # 4) Thompson‑sampled curve ------------------------------
    theta_ts = sample_theta(mu, Sig)
    d_ts = static_vec @ theta_ts[:len(static_vec)] + PB @ theta_ts[len(static_vec):]

    # 5) Profit function & argmax ----------------------------
    profit_ts = (p_grid - COST_PER_UNIT) * np.maximum(d_ts, 0)
    p_star = p_grid[np.argmax(profit_ts)]
    max_profit = np.max(profit_ts)

    # 6) Plot -------------------------------------------------
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))

    # Panel 1 – demand curves
    ax1.plot(p_grid, d_mean, color="black", label="Posterior mean")
    ax1.fill_between(p_grid, lower, upper,
                     color="steelblue", alpha=0.3, label="95% CI")
    ax1.plot(p_grid, d_ts, color="green", label="TS realisation")
    ax1.set_xlabel("Price"); ax1.set_ylabel("Quantity")
    ax1.set_title(f"Demand curves at τ={idx}")
    ax1.legend()

    # Panel 2 – profit curve + optimal price
    ax2.plot(p_grid, profit_ts, color="red", label="Profit (TS sample)")
    ax2.axvline(p_star, color="black", linestyle="--", label=f"p*={p_star:.1f}")
    ax2.set_xlabel("Price"); ax2.set_ylabel("Profit")
    ax2.set_title("Objective function")
    ax2.legend()

    plt.tight_layout()
    fname = f"ts_realisation_tau{idx}.png"
    plt.savefig(os.path.join(plots_dir, fname), dpi=150)
    plt.show()
    print(f"✓ Figure saved to {os.path.join(plots_dir, fname)}")

# Example call
# plot_ts_realisation(idx=5000, use_final_posterior=False, context="row")

In [ ]:
# ════════════════════════════════════════════════════════════════
#  SNAPSHOT MODEL‑2 RESULTS  →  results_model_2
#  • Run immediately after Model‑2’s training & persistence cells.
#  • No re‑execution of the model; just bundle what’s already there.
#  • Automatic fallback: reloads the latest phase1_* folder on disk
#    if the expected in‑memory objects are missing.
# ════════════════════════════════════════════════════════════════
import os
from glob import glob
import pprint
import numpy as np
import pandas as pd

# ────────────────────────────────────────────────────────────────
# 1.  Check whether Model‑2 artefacts are still in RAM
# ────────────────────────────────────────────────────────────────
_have_mem = all(name in globals() for name in [
    "logs",               # big dict created in Block 4
    "df_logs",            # pretty transaction DataFrame (Block 5)
    "theta_mean_trace",   # posterior path
    "post_mean_final",    # final μ
    "post_cov_final",     # final Σ
])

if _have_mem:
    print("✓ Model‑2 artefacts found in memory – using them directly.")
    model2_folder = globals().get("base_dir", "") or "‑in‑memory‑run‑"
    df_logs2      = df_logs
    theta2        = theta_mean_trace
    mu2           = post_mean_final
    Sigma2        = post_cov_final
    meta2         = {
        "feature_cols" : logs["feature_cols"],
        "price_bounds" : logs["price_bounds"],
        "sigma2"       : logs["sigma2"],
        "cost_per_unit": logs["cost_per_unit"],
        "folder"       : model2_folder
    }

else:
    # ────────────────────────────────────────────────────────────
    # 2.  Fallback – reload the *newest* “phase1_*” folder
    # ────────────────────────────────────────────────────────────
    try:
        load_phase1_results          # helper defined in Block 5
    except NameError as err:
        raise NameError("Model‑2 objects not in RAM and helper "
                        "`load_phase1_results()` is missing. "
                        "Run Model‑2’s persistence block first.") from err

    phase_dirs = sorted(
        glob(os.path.join("results", "phase1_*")),
        key=os.path.getmtime
    )
    if not phase_dirs:
        raise FileNotFoundError("No phase1_* folders found in ./results/. "
                                "Run Model‑2 first.")
    model2_folder = phase_dirs[-1]   # latest folder
    print(f"ℹ Model‑2 artefacts reloaded from  {model2_folder}")
    df_logs2, theta2, mu2, Sigma2, meta2 = load_phase1_results(model2_folder)

# ────────────────────────────────────────────────────────────────
# 3.  Bundle everything neatly into `results_model_2`
# ────────────────────────────────────────────────────────────────
results_model_2 = {
    "folder"     : model2_folder,
    "df_logs"    : df_logs2,      # per‑transaction DataFrame
    "theta_trace": theta2,        # (N, d) posterior mean over time
    "mu_final"   : mu2,           # final posterior mean
    "Sigma_final": Sigma2,        # final posterior covariance
    "meta"       : meta2          # misc. meta‑info
}

print("\nresults_model_2 is now available with keys:")
pprint.pp(results_model_2.keys())
print(f"\n↳ Total profit recorded by Model‑2 = "
      f"{results_model_2['df_logs']['profit'].sum():,.2f}")
